# Universo / Target

## Definición de target

Target = 1 cuando el alumno no curso ni rindió finales por 2 ciclos lectivos consecutivos y no haya finalizado la carrera

## Dataset a armar

ejemplo

| Alumno | Ciclo | Target |
| --- | --- | --- |
| 1 | 2010 | 0 |
| 1 | 2011 | 0 |
| 1 | 2012 | 1 |
| 1 | 2013 | 1 |
| 2 | 2012 | 0 |
| 2 | 2013 | 0 |
| 2 | 2014 | 0 |
| 2 | 2015 | 0 |

- ciclo minimo es el año de ingreso del alumno
- ciclo maximo es el año de egreso del alumno o del ultimo registro (cursadas o finales) + 2




In [1]:
import pandas as pd
import numpy as np

In [2]:
import seaborn as sns

In [3]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


## Datos de entrada

In [4]:
cur = pd.read_pickle('../data/raw_cleaned/curdf_03.pkl')
fin = pd.read_pickle('../data/raw_cleaned/findf_03.pkl')
curcomp = pd.read_pickle('../data/raw_cleaned/curdf_complementario_03.pkl')
dat = pd.read_pickle('../data/raw_cleaned/datdf_03.pkl')
df_ult = pd.read_pickle('../data/processed/dataset_intermedio_primeros_y_ultimos_registros_03.pkl')
df = pd.read_pickle('../data/processed/df_agrupado_target_03.pkl')


In [5]:
print('cur shape: ',cur.shape)
print('cur_comp shape: ',curcomp.shape)
print('datdf shape: ',dat.shape)
print('fin shape: ',fin.shape)
print('df_ultimos_registros shape: ',df_ult.shape)
print('df_agrupado_target shape: ',df.shape)

cur shape:  (464280, 9)
cur_comp shape:  (165810, 9)
datdf shape:  (20016, 5)
fin shape:  (424410, 8)
df_ultimos_registros shape:  (20016, 8)
df_agrupado_target shape:  (20016, 11)


### Planificacion de Datasets

In [6]:
cur.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas,dataset
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Noche,2006,1.0,1,cursadas
4497,10035,K3051,952028,NaN,Noche,2007,1.0,1,cursadas
4492,10035,K3001,952030,NaN,Mañana,2006,1.0,1,cursadas
4498,10035,K3071,952031,NaN,Noche,2007,1.0,1,cursadas
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Noche,2008,1.0,1,cursadas


In [8]:
curcomp.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas,dataset
20181,10035,S/D,950309,S/D,S/D,2006.0,1,1,cursadas_complementario
20173,10035,S/D,950611,S/D,S/D,2004.0,1,1,cursadas_complementario
20166,10035,S/D,950701,S/D,S/D,2003.0,1,1,cursadas_complementario
20167,10035,S/D,950702,S/D,S/D,2003.0,1,1,cursadas_complementario
20176,10035,S/D,950703,S/D,S/D,2005.0,1,1,cursadas_complementario


In [9]:
dat.head()

,codigo_alumno,sexo,anio_nacimiento,anio_ingreso,codigo_carrera_ingreso
1,53,M,1974,1999,K
2,58,M,1974,1995,K
3,72,M,1974,1995,K
4,86,M,1974,1999,K
10,313,M,1974,2001,K


In [10]:
fin.head()

,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,cant_finales,dataset
20181,10035,950309,2006.0,2.0,NaN,No Aprobada,1.0,finales
20182,10035,950309,2006.0,4.0,NaN,Aprobada,2.0,finales
20183,10035,950310,2006.0,7.0,NaN,Aprobada,1.0,finales
20173,10035,950611,2004.0,4.0,NaN,Aprobada,1.0,finales
20166,10035,950701,2003.0,4.0,NaN,Aprobada,1.0,finales


In [23]:
# contar valores dentro del groupby 
aux = fin.groupby(['codigo_alumno','codigo_materia'])['nota_txt'].value_counts().reset_index(name='count')
aux = aux[(aux['count'] > 1)&(aux.nota_txt == 'Aprobada')].copy()
aux

,codigo_alumno,codigo_materia,nota_txt,count
735,10142,950309,Aprobada,2
3588,10966,950309,Aprobada,2
4664,11444,952036,Aprobada,2
5168,11751,951603,Aprobada,2
6035,12006,950704,Aprobada,2
...,...,...,...,...
399016,9025,952072,Aprobada,2
399325,9291,952025,Aprobada,2
400262,9505,952034,Aprobada,2
400372,9513,952039,Aprobada,2


In [25]:
aux2 = pd.merge(aux,fin, on=['codigo_alumno','codigo_materia','nota_txt'], how='left')

### criterio

- se encontraron datos como que los alumnos se sacaban un 10 y despues de 2 años rendiand de nuevo y se sacanab  un 4. ademas de ser absurdo y debido a la cantidad de errores y sin forma de poder corregirlo por ahora el proyecto queda pausado hasta que se pueda corregir los datos